# Lab 2

<b>Class:</b> MSDS 7331 Data Mining
<br> <b>Dataset:</b> Belk Endowment Educational Attainment Data 

<h1 style="font-size:150%;"> Teammates </h1>
Maryam Shahini
<br> Murtada Shubbar
<br> Michael Toolin
<br> Steven Millett

In [2]:
#Set global variables
#Variables for file and school informaiton

YEARS = ['2014', '2015', '2016', '2017']
SCHOOLS = ['High','Middle','Elementary']

#Number of features we will be selecting for feature selection

N_FEATURES_OPTIONS = [2, 25 , 50]

#Alpha and C we will be using for our classifiers

C_OPTIONS = [1e-2, 1e-1, 1e0, 1e1, 1e2]

#Import data all necessary libraries we will be using in our estimation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import sklearn
import statistics


#from umap.umap_ import UMAP
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile, RFE

from sklearn.preprocessing import StandardScaler, Binarizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import accuracy_score, confusion_matrix
from IPython.display import display, HTML

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.naive_bayes import MultinomialNB


%matplotlib inline

# 1.a Data Preparation
10 points - Deﬁne and prepare your class variables. Use proper variable 
representations (int, ﬂoat, one-hot, etc.). Use pre-processing methods (as needed) for
dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for 
the analysis.

# 1.b Data Preparation
5 points - Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

In [3]:
# The 2017 Public Schools Machine Learning 
# Date Set is being used throughout this 
# analysis.  The _ML suffix is removed to less 
# name space size
# Load Full Public School Data Frames for each year

school_data = pd.DataFrame()

for year in YEARS:
    #Load public school master file
    temp_year = pd.read_csv('../Data/'+str(year)+'/Machine Learning Datasets/PublicSchools'+str(year)+'_ML.csv', low_memory=False)
    
    #Iterate through adding and merging school data based on school type
    for grade in SCHOOLS:
        grade_temp_year = pd.read_csv('../Data/'+year+'/Machine Learning Datasets/Public'+grade+'Schools'+year+'_ML.csv', low_memory=False)
        cols_to_use = grade_temp_year.columns.difference(temp_year.columns)
        cols_to_use = np.append(cols_to_use,'unit_code')
        temp_year = pd.merge(temp_year, grade_temp_year[cols_to_use],left_index=True, right_index=True, on='unit_code',how='left' )
    
    #Add year column and concatonating all data together
    temp_year['Year']=year
    school_data = pd.concat([school_data,temp_year],ignore_index=True)

C:\Users\Bahr\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
#This is the critical threshold
CRITICAL_NA = .75

#With this we check if the column is less than 75% non-NA, if it is greater than 75% non-NA
#We replace the NA with the median of the column, otherwise we replace the value with 0

imputed_school_data = school_data.apply(lambda col: col.fillna(0) if col.count()/col.shape[0]<CRITICAL_NA else col.fillna(col.median()),axis=0)




In [14]:

#ed = imputed_school_data.copy()

#ed.to_csv('C:\\Users\\Bahr\\Desktop\\SMU_DS_Summer_2018\\Data_Mining\\Assignments\\Lab TWO\\MSDS7331_NorthCarolinaDataset\\Lab2\\RAW.csv')

#ED is processed using Alteryx 


#reading in new dataset. 
SAT_Filtered = pd.DataFrame()
SAT_Filtered = pd.read_csv('../Data/School Datasets/Alteryx_Product.csv', low_memory=False)


SAT_Filtered

#SAT_SCORE_ZERO	Boolean: If SAT equals zero. Used it to test if anyone participated but produced zero scores. 
#SAT_Score_above1000	 Boolean: SAT score above 1000 or not to get into a decent college in NC. 
#SAT_participation_number	 Estimate of number of students that took the SAT
#EST_Student_College_NO	 Estimate of the number of students who didn't attempt the SAT
#Student_Num_College_Ready_SAT	 Estimate of the number of students that can apply to college without SAT score hinderance from that school. 


,Row,Year,avg_daily_attend_pct,lea_federal_perpupil_num,lea_local_perpupil_num,lea_salary_expense_pct,lea_services_expense_pct,lea_state_perpupil_num,lea_supplies_expense_pct,lea_total_expense_num,...,EOGReadingGr4_CACR_SWD,sat_avg_score_num,sat_participation_pct,student_num,SAT_SCORE_ZERO,SAT_Score_above1000,SAT_participation_number,EST_Student_College_NO,Student_Num_College_Ready_SAT,Unnamed: 272
0,0,2014,0.872,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,0.0,925,0.12,78,0,0,9.36,68.64,0.00,NaN
1,1,2014,0.957,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,0.0,979,0.63,539,0,0,339.57,199.43,0.00,NaN
2,2,2014,0.954,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,30.0,884,0.70,547,0,0,382.90,164.10,0.00,NaN
3,3,2014,0.943,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,0.0,824,0.45,800,0,0,360.00,440.00,0.00,NaN
4,4,2014,0.944,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,14.3,962,0.55,664,0,0,365.20,298.80,0.00,NaN
5,5,2014,0.958,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,0.0,1024,0.60,330,0,1,198.00,132.00,198.00,NaN
6,6,2014,0.926,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,0.0,1008,0.64,1192,0,1,762.88,429.12,762.88,NaN
7,7,2014,0.947,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,0.0,983,0.32,492,0,0,157.44,334.56,0.00,NaN
8,8,2014,0.952,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,10.0,959,0.42,628,0,0,263.76,364.24,0.00,NaN
9,9,2014,0.960,1006.40,1845.78,0.613,0.078,5176.41,0.086,8028.59,...,6.3,883,0.62,621,0,0,385.02,235.98,0.00,NaN


# Get list of highly correlated features

In [6]:
# corr_matrix = imputed_school_data.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find index of feature columns with correlation greater than 0.95
# to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]

# print(to_drop)

['GraduationRate_5yr_Female', 'Math Course Rigor Score', 'lea_highqual_class_all_pct', 'lea_highqual_class_pct', 'st_highqual_class_pct', 'st_total_specialized_courses']


# 2.a Modeling and Evaluation
Using the right evaluation metric for classification system is crucial. Otherwise, it may results in thinking that the model is performing well but in reality, it doesn’t.

There are two tasks in this section of “NC Educational Data” project:

The first task is to predict a binary classification target, either if the average SAT score of each school is good enough to gets the student to the North Carolina Universities or not. The SAT is a standardized test widely used for college admissions in the United States. For this purpose we have a cut off 1200 out of 1600. The second task is to predict if the crime per 100 students at each school level is higher than the LEA level or not. After considering all evaluation metrics for classification systems, we ended up using ROC Curve. Area under ROC Curve (or AUC for short) is a performance metric for binary classification problems.

In fact, a ROC curve can be used to select a threshold for a classifier which maximizes the true positives, while minimizing the false positives.

We usually use ROC when both classes detection are important. Here, our models should be able to decrease both false positive rate (which is identifying the schools with enough good average SAT score for getting admission in different universities) and also decreasing the false negative rate (which is detecting schools with not good average SAT scores).

The same for the second task, it is important to decrease both false positive and false negative rates.

The AUC represents a model’s ability to discriminate between positive and negative classes. An area of 1.0 represents a model that made all predictions perfectly. An area of 0.5 represents a model as good as random. Most classifiers have AUCs that fall somewhere between these two values. Therefore, the overall model performances can be compared by considering the AUC.

# 2.b Modeling and Evaluation
10 points - Choose the method you will use for dividing your data into training and why testing splits (i.e., are you using Stratiﬁed 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate.

# Binary Classification model

In [42]:
# split X and y into test and train sets. We still want
# to do this for external Cross Validation

crime_imputed_school_data = imputed_school_data

crime_imputed_school_data['local_crime_greater'] = crime_imputed_school_data.apply(lambda each_row: 1 if (each_row['crime_per_c_num']-each_row['lea_crime_per_c_num'])<0 else 0,axis=1)

#split data into X and y dataframes

y_crime = crime_imputed_school_data['local_crime_greater']

#Removed SPG Grade and unit code(which is primary key for school data table)
 
X_crime = imputed_school_data[school_data.columns.drop(list(school_data.filter(regex='crime|unit_code|lea|LEA|^st\_')))]

X_crime_train, X_crime_test, y_crime_train, y_crime_test = train_test_split(X_crime, y_crime, test_size=.2)


# 2.c Modeling and Evaluation
20 points - Create three different classification/regression models (e.g., random forest, KNN, and SVM). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric.

In [43]:
k_fold = KFold(n_splits=10,shuffle=True)

#This creates the template for the pipeline
# This creates a basic pipeline where we will 
# test for dementionality reduction, scaling,
# and classification.


pipe = Pipeline([ ('reduce_dim',SelectKBest(chi2)),
                  ('scale', StandardScaler()), 
                  ('clf', GradientBoostingRegressor())])

In [44]:
 param_grid = [
    {
         'reduce_dim__k': N_FEATURES_OPTIONS,
         'clf__n_estimators': C_ESTIMATORS, 
         'clf__max_depth': C_DEPTH,
     }
]


# # This will test the parameter dict against our 
# # pipeline

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=k_fold,n_jobs=-1, verbose=1 )


# # Here we are training the model, this is 
# # what takes the most amount of time to run
crime_GradientBoost_model = grid_search.fit(X_crime_train, y_crime_train)


y_crime_score = grid_search.predict(X_crime_test)

print(roc_auc_score(y_crime_test, y_crime_score))


Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  7.0min finished


0.7043625376861664


In [45]:
pipe.set_params(**crime_GradientBoost_model.best_params_)
pipe.fit(X_crime_train, y_crime_train)


Pipeline(memory=None,
     steps=[('reduce_dim', SelectKBest(k='all', score_func=<function chi2 at 0x7ff484499620>)), ('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, ma...rs=50, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False))])

In [46]:
coef = pipe.steps[2][1].feature_importances_

mask = pipe.steps[0][1].get_support()
new_features=[]
feature_names=list(X_crime_train.columns.values)
for bool, feature in zip(mask, feature_names):
    if bool:
        new_features.append(feature)

#Creates a new dataframe with the coefficients and the 
predicted_data = pd.DataFrame(data=coef,index=new_features,columns=['Influence'])
print("The top 20 features that influence SPG Grade are the following")



display(predicted_data.sort_values(by='Influence', ascending=False)[0:20])

The top 20 features that influence SPG Grade are the following


,Influence
short_susp_per_c_num,0.089424
avg_daily_attend_pct,0.024873
SPG Score,0.019061
TwoOrMorePct,0.017637
BlackFemalePct,0.017281
EOG/EOCSubjects_CACR_All,0.016024
EOG/EOCSubjects_CACR_Hispanic,0.015840
books_per_student,0.013949
EOG/EOCSubjects_CACR_White,0.013150
EOGScienceGr8_GLP_SWD,0.011988


In [76]:
 param_grid = [
    {
         'reduce_dim__k': N_FEATURES_OPTIONS,
         'clf': [AdaBoostClassifier()],
         'clf__n_estimators': C_ESTIMATORS

     }
]


# # This will test the parameter dict against our 
# # pipeline

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=k_fold,n_jobs=-1, verbose=1 )


# # Here we are training the model, this is 
# # what takes the most amount of time to run
crime_ADABoost_model = grid_search.fit(X_crime_train, y_crime_train)

y_crime_score = grid_search.predict(X_crime_test)

print(roc_auc_score(y_crime_test, y_crime_score))


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.5min finished


0.6369818484422948


In [77]:
print(crime_ADABoost_model.best_params_)

{'clf': AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), 'clf__n_estimators': 50, 'reduce_dim__k': 'all'}


In [78]:
 param_grid = [
    {
         'reduce_dim__k': N_FEATURES_OPTIONS,
         'clf': [RandomForestClassifier()],
         'clf__n_estimators': C_ESTIMATORS, 
         'clf__max_depth': C_DEPTH,
     }
]


# # This will test the parameter dict against our 
# # pipeline

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=k_fold,n_jobs=-1, verbose=1 )


# # Here we are training the model, this is 
# # what takes the most amount of time to run
crime_RandomForest_model = grid_search.fit(X_crime_train, y_crime_train)

y_crime_score = grid_search.predict(X_crime_test)

print(roc_auc_score(y_crime_test, y_crime_score))

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   58.1s finished


0.6155341446923598


In [79]:
print(crime_RandomForest_model.best_params_)
print(crime_RandomForest_model.multimetric_)

{'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'clf__max_depth': 5, 'clf__n_estimators': 50, 'reduce_dim__k': 'all'}
False


In [80]:
 param_grid = [
    {
         'reduce_dim__k': N_FEATURES_OPTIONS,
         'clf': [KNeighborsClassifier()],
         'clf__n_neighbors': C_NEIGHBORS, 
     }
]


# # This will test the parameter dict against our 
# # pipeline

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=k_fold,n_jobs=-1, verbose=1 )


# # Here we are training the model, this is 
# # what takes the most amount of time to run
crime_KNearest_model = grid_search.fit(X_crime_train, y_crime_train)

y_crime_score = grid_search.predict(X_crime_test)

print(roc_auc_score(y_crime_test, y_crime_score))

Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed: 10.9min finished


0.6263636422738856


In [81]:
print(crime_KNearest_model.best_params_)

{'clf': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=40, p=2,
           weights='uniform'), 'clf__n_neighbors': 40, 'reduce_dim__k': 100}


# Reduced scope model

In [53]:
X_crime_reduced = X_crime[X_crime.columns.drop(list(X_crime.filter(regex='[Aa]sian|[Hh]ispanic|[Rr]ace|[Bb]lack|[Mm]inority|[Tw]wo[Oo]r[Mm]ore|[Ii]ndian|[Ww]hite')))]
X_crime_reduced_train, X_crime_reduced_test, y_crime_reduced_train, y_crime_reduced_test = train_test_split(X_crime_reduced, y_crime, test_size=.2)

In [54]:
print(X_crime_reduced.columns.values)

['00_Size' '01_Size' '02_Size' '03_Size' '04_Size' '05_Size' '06_Size'
 '07_Size' '08_Size' '09_Size' '10_Size' '11_Size' '12_Size'
 '4-Year Cohort Graduation Rate Score' 'AAVC_Concentrator_Ct' 'ACT Score'
 'ACT WorkKeys Score' 'ACTCompositeScore_UNCMin_AIG'
 'ACTCompositeScore_UNCMin_EDS' 'ACTCompositeScore_UNCMin_Female'
 'ACTCompositeScore_UNCMin_LEP' 'ACTCompositeScore_UNCMin_Male'
 'ACTCompositeScore_UNCMin_SWD' 'ACTEnglish_ACTBenchmark_AIG'
 'ACTEnglish_ACTBenchmark_EDS' 'ACTEnglish_ACTBenchmark_Female'
 'ACTEnglish_ACTBenchmark_LEP' 'ACTEnglish_ACTBenchmark_Male'
 'ACTEnglish_ACTBenchmark_SWD' 'ACTMath_ACTBenchmark_AIG'
 'ACTMath_ACTBenchmark_All' 'ACTMath_ACTBenchmark_EDS'
 'ACTMath_ACTBenchmark_Female' 'ACTMath_ACTBenchmark_LEP'
 'ACTMath_ACTBenchmark_Male' 'ACTMath_ACTBenchmark_SWD'
 'ACTReading_ACTBenchmark_AIG' 'ACTReading_ACTBenchmark_All'
 'ACTReading_ACTBenchmark_EDS' 'ACTReading_ACTBenchmark_Female'
 'ACTReading_ACTBenchmark_LEP' 'ACTReading_ACTBenchmark_Male'
 'ACTRead

In [55]:
 param_grid = [
    {
         'reduce_dim__k': N_FEATURES_OPTIONS,
         'clf__n_estimators': C_ESTIMATORS, 
         'clf__max_depth': C_DEPTH,
     }
]


# # This will test the parameter dict against our 
# # pipeline

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=k_fold,n_jobs=-1, verbose=1 )


# # Here we are training the model, this is 
# # what takes the most amount of time to run
crime_reduced_GradientBoost_model = grid_search.fit(X_crime_reduced_train, y_crime_reduced_train)


y_crime_reduced_score = grid_search.predict(X_crime_reduced_test)

print(roc_auc_score(y_crime_reduced_test, y_crime_reduced_score))


Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  5.1min finished


0.6750940987337503


In [56]:
pipe.set_params(**crime_reduced_GradientBoost_model.best_params_)
pipe.fit(X_crime_reduced_train, y_crime_reduced_train)

Pipeline(memory=None,
     steps=[('reduce_dim', SelectKBest(k='all', score_func=<function chi2 at 0x7ff484499620>)), ('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, ma...rs=50, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False))])

In [57]:
coef = pipe.steps[2][1].feature_importances_

mask = pipe.steps[0][1].get_support()
new_features=[]
feature_names=list(X_crime_reduced_train.columns.values)
for bool, feature in zip(mask, feature_names):
    if bool:
        new_features.append(feature)

#Creates a new dataframe with the coefficients and the 
predicted_data = pd.DataFrame(data=coef,index=new_features,columns=['Influence'])
print("The top 20 features that influence SPG Grade are the following")



display(predicted_data.sort_values(by='Influence', ascending=False)[0:20])

The top 20 features that influence SPG Grade are the following


,Influence
short_susp_per_c_num,0.095112
avg_daily_attend_pct,0.046270
EOG/EOCSubjects_CACR_All,0.027405
SPG Score,0.019949
student_num,0.019401
stud_internet_comp_num,0.016367
EVAAS Growth Score,0.015583
advance_dgr_pct,0.015271
books_per_student,0.014892
long_susp_per_c_num,0.013610


# 2.d Modeling and Evaluation
10 points - Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

In [25]:
# #Run this to load the model from the save file

# from sklearn.externals import joblib
# grid_search = joblib.load('savedBestModel.pkl')


# # Loads all parameters run into a dict 

# params = np.array(grid_search.cv_results_['params'])


# # Loads all mean test scores into an array

# mean_scores = np.array(grid_search.cv_results_['mean_test_score'])

In [26]:
# # Assigns all models to an array

# classifier_labels=['SVC','LogisticRegression','SGDClassifier']


# # Creates an empty dataframe that is to be
# # filled with the mean test accuracy by C global
# # variable and the different classifiers

# classifier_temp = pd.DataFrame(columns=classifier_labels,index=C_OPTIONS,
#                                data=np.linspace(.1,.25,num=len(C_OPTIONS)*len(classifier_labels)).reshape(len(C_OPTIONS),len(classifier_labels)))
# classifier_temp.fillna(0,inplace=True)

# for i, (param, score) in enumerate(zip(params, mean_scores)):
#     C = param['clf__C'] if 'clf__C' in param else param['clf__alpha']
#     class_state = str(param['clf']).split('(')[0]
#     if classifier_temp.at[C,class_state] < score:
#         classifier_temp.at[C,class_state] = score


# # Printing a grid of the best accuracies
        
# display(classifier_temp.transpose())   


# # Print a line plot which shows the best 
# # accuracies
 
# classifier_temp.plot(logx=True,ylim=(0,1),figsize=(14,10),title='Accuracy by Classifier'); 

In [27]:
# # Assigns all reduction models to an array

# reduce_labels=['NMF','PCA','SelectKBest']


# # Translates the N Features array
# # to an array full of string

# temp_N_FEATURES_OPTIONS = [str(r) for r in N_FEATURES_OPTIONS]
# temp_N_FEATURES_OPTIONS=temp_N_FEATURES_OPTIONS+['None']


# # Creates an empty dataframe that is to be
# # filled with the mean test accuracy by N Features
# # variable and the different feature reduction models

# reduce_temp = pd.DataFrame(columns=reduce_labels,index=temp_N_FEATURES_OPTIONS,
#                                data=np.linspace(.1,.25,num=len(temp_N_FEATURES_OPTIONS)*len(reduce_labels)).reshape(+len(temp_N_FEATURES_OPTIONS),len(reduce_labels)))


# for i, (param, score) in enumerate(zip(params, mean_scores)):
#     trigger=0
#     reduce_state = str(param['reduce_dim']).split('(')[0]
#     if 'reduce_dim__k' in param:
#         N_FEAT = str(param['reduce_dim__k'])
#         trigger=1
#     elif 'reduce_dim__n_components' in param:
#         N_FEAT = str(param['reduce_dim__n_components'])
#         trigger=1
#     else:
#         if reduce_temp.at['None','NMF'] < score:
#             reduce_temp.at['None','NMF'] = score
#             reduce_temp.at['None','SelectKBest'] = score
#     if trigger == 1:
#         if reduce_temp.at[N_FEAT,reduce_state] < score:
#             reduce_temp.at[N_FEAT,reduce_state] = score

            
# # Printing a grid of the best accuracies

# display(reduce_temp.transpose())


# # Print a bar plot which shows the best 
# # accuracies

# reduce_temp.plot(kind='bar',ylim=(0,1),figsize=(14,10),title='Accuracy by Feature Selection',rot=0);           

In [ ]:
# print('The Index of the best model is',grid_search.best_index_)
# print('The parameters of the best model is')
# display(grid_search.best_params_)
# print('The accuracy of the best model is',round(grid_search.best_score_*100,4))

# 2.e Modeling and Evaluation
10 points - Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference signiﬁcant with 95% conﬁdence? Use proper statistical comparison methods.

# 2.f Modeling and Evaluation
10 points - Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classiﬁcation task.

# Deployment
5 points - How useful is yolur model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

# Exceptional Work
10 points - You have free reign to provide additional modeling. 
One idea: grid search parameters in a parallelized fashion and visualize the 
performances across attributes. Which parameters are most signiﬁcant for making a 
good model for each classiﬁcation algorithm?